In [ ]:
""" %pip install --quiet hvplot
%pip install --quiet holoviews[all]
%pip install --quiet jupyter_bokeh """

In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import plotly.graph_objects as go

import holoviews as hv
from holoviews import opts, dim


In [ ]:
# Load data

path_2022 = "../data/movilidad_provincias_2022.csv"
path_2023 = "../data/movilidad_provincias_2023.csv"
path_2024 = "../data/movilidad_provincias_2024.csv"

original_data_2022 = pd.read_csv(path_2022, sep=",")
original_data_2023 = pd.read_csv(path_2023, sep=",")
original_data_2024 = pd.read_csv(path_2024, sep=",")

df_2022 = original_data_2022.copy()
df_2023 = original_data_2023.copy()
df_2024 = original_data_2024.copy()

df = pd.concat([df_2022, df_2023, df_2024])

In [ ]:
df.info()
df.shape

In [ ]:
# Con estos datos, generamos un diagrama de flujo de los viajeros de la peninsula hacia las islas
#%pip install networkx

In [ ]:
import networkx as nx

# Filtrar datos para viajeros de la península a las islas
codigos_provincias_insulares = [7, 35, 38] # Baleares, Las Palmas y Santa Cruz de Tenerife
viajeros_islas = df[df['provincia_origen'].isin(codigos_provincias_insulares)]

# Crear un grafo dirigido
G = nx.DiGraph()

# Añadir nodos
G.add_nodes_from(viajeros_islas['provincia_origen_name'].unique(), isla=True)
G.add_nodes_from(viajeros_islas['provincia_destino_name'].unique(), isla=False)

# Añadir aristas
for _, row in viajeros_islas.iterrows():
    G.add_edge(row['provincia_origen_name'], row['provincia_destino_name'], weight=row['viajeros'])

# Dibujar el grafo
plt.figure(figsize=(10, 10))
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_size=3000, node_color=['blue' if G.nodes[n]['isla'] else 'red' for n in G.nodes])
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
plt.show()

In [ ]:
# DIAGRAMA DE SANKEY
import plotly.graph_objects as go
import pandas as pd

# Prepare data for Sankey diagram
source = []
target = []
value = []
node_labels = []

# Get unique provinces for node indexing
all_provinces = pd.concat([
    viajeros_islas['provincia_origen_name'],
    viajeros_islas['provincia_destino_name']
]).unique()

# Create node index mapping
node_map = {province: idx for idx, province in enumerate(all_provinces)}

# Create links data
for _, row in viajeros_islas.iterrows():
    source.append(node_map[row['provincia_origen_name']])
    target.append(node_map[row['provincia_destino_name']])
    value.append(row['viajeros'])

# Create node colors
node_colors = ['blue' if provincia in viajeros_islas['provincia_origen_name'].unique() 
               else 'red' for provincia in all_provinces]

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = all_provinces,
        color = node_colors
    ),
    link = dict(
        source = source,
        target = target,
        value = value
    )
)])

# Update layout
fig.update_layout(
    title_text="Flujo de Viajeros entre Islas y Península",
    font_size=10,
    height=800
)

fig.show()

In [ ]:
import holoviews as hv

# Set up holoviews
hv.extension('bokeh')
hv.output(size=300)

# Prepare data structure for Sankey diagram
sources = []
targets = []
values = []

# Create nodes mapping (provinces to indices)
provinces = pd.concat([
    viajeros_islas['provincia_origen_name'],
    viajeros_islas['provincia_destino_name']
]).unique()
node_map = {province: idx for idx, province in enumerate(provinces)}

# Create links data
for _, row in viajeros_islas.iterrows():
    sources.append(node_map[row['provincia_origen_name']])
    targets.append(node_map[row['provincia_destino_name']]) 
    values.append(row['viajeros'])

# Create nodes dataset
nodes = hv.Dataset(enumerate(provinces), 'index', 'label')

# Create Sankey diagram
sankey = hv.Sankey((list(zip(sources, targets, values)), nodes))

# Format visualization
sankey.opts(
    opts.Sankey(
        label_position='outer',
        edge_color=dim('source').str(),
        node_color=dim('index').str(),
        labels='label',
        width=800,
        height=500,
        cmap='Category20',
        title='Flujo de Viajeros entre Islas y Península'
    )
)

sankey

In [ ]:
# Filtrar datos para viajeros de la península a las islas
""" codigos_provincias_insulares = [7, 35, 38] # Baleares, Las Palmas y Santa Cruz de Tenerife
viajeros_islas = df[df['provincia_origen'].isin(codigos_provincias_insulares)]


# Crear un diagrama de flujo
import hvplot.pandas

# Asumimos que las columnas relevantes son 'origen', 'destino' y 'viajeros'
diagrama_flujo = viajeros_islas.hvplot.sankey(
    'origen', 'destino', 'viajeros', 
    title='Diagrama de flujo de viajeros de la península a las islas'
)

diagrama_flujo """